In [ ]:
!git clone https://github.com/NormBill/Cross_Market_Recommendation.git

fatal: destination path 'Cross_Market_Recommendation' already exists and is not an empty directory.


In [ ]:
import os
import urllib.request as urlreq

tgt_markets = ['ca']
tgt_cat = 'Electronics'

fix_url = 'https://ciir.cs.umass.edu/downloads/XMarket/FULL/'
orig_data_dl = 'DATA2/orig_data'
proc_data_out = 'DATA2/proc_data'
if not os.path.exists(orig_data_dl):
    os.makedirs(orig_data_dl)
if not os.path.exists(proc_data_out):
    os.makedirs(proc_data_out)

for tgt_market in tgt_markets:
    cur_url = f'{fix_url}{tgt_market}/{tgt_cat}/ratings_{tgt_market}_{tgt_cat}.txt.gz'
    urlreq.urlretrieve(cur_url, f'{orig_data_dl}/ratings_{tgt_market}_{tgt_cat}.txt.gz')
    print(f'Done: {cur_url}')
    cur_url_review = f'{fix_url}{tgt_market}/{tgt_cat}/reviews_{tgt_market}_{tgt_cat}.json.gz'
    urlreq.urlretrieve(cur_url_review, f'{orig_data_dl}/reviews_{tgt_market}_{tgt_cat}.json.gz')
    print(f'Done: {cur_url_review}')

Done: https://ciir.cs.umass.edu/downloads/XMarket/FULL/ca/Electronics/ratings_ca_Electronics.txt.gz
Done: https://ciir.cs.umass.edu/downloads/XMarket/FULL/ca/Electronics/reviews_ca_Electronics.json.gz


In [ ]:
import numpy as np
import torch

In [ ]:
import pandas as pd

# us_ratings_file = "/content/Cross_Market_Recommendation/us_user_part0.csv"
ca_ratings_file = "/content/Cross_Market_Recommendation/ca_user_data.csv"
# sg_ratings_file = "/content/Cross_Market_Recommendation/sg_user_data.csv"
# us_df = pd.read_csv(us_ratings_file, skiprows=1, header=None, names=["userId", "itemList"])
ca_df = pd.read_csv(ca_ratings_file, skiprows=1, header=None, names=["userId", "itemList"])
# sg_df = pd.read_csv(sg_ratings_file, skiprows=1, header=None, names=["userId", "itemList"])
# us_df.info()
ca_df.info()
# sg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422228 entries, 0 to 422227
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   userId    422228 non-null  object
 1   itemList  422228 non-null  object
dtypes: object(2)
memory usage: 6.4+ MB


In [ ]:
# print("US DataFrame - First Row:")
# print(us_df.head(1))

print("CA DataFrame - First Row:")
print(ca_df.head(1))

# print("SG DataFrame - First Row:")
# print(sg_df.head(1))

CA DataFrame - First Row:
                         userId                      itemList
0  AG7XIRXSJ6MOXRGBI7QTKRPL2FSQ  ['B00C2P61FO', 'B00NEYHIHM']


In [ ]:
import os
import urllib.request as urlreq
import gzip
import json

# us_ratings_file = f'{orig_data_dl}/ratings_us_{tgt_cat}.txt.gz'

# us_rating_df = pd.read_csv(us_ratings_file, compression='gzip', header=None, sep=' ', names=["userId", "itemId", "rate", "date"] )

# us_items_set = set(us_rating_df['itemId'].unique())

ca_ratings_file = f'{orig_data_dl}/ratings_ca_{tgt_cat}.txt.gz'

ca_rating_df = pd.read_csv(ca_ratings_file, compression='gzip', header=None, sep=' ', names=["userId", "itemId", "rate", "date"] )

ca_items_set = set(ca_rating_df['itemId'].unique())

# sg_ratings_file = f'{orig_data_dl}/ratings_sg_{tgt_cat}.txt.gz'

# sg_rating_df = pd.read_csv(sg_ratings_file, compression='gzip', header=None, sep=' ', names=["userId", "itemId", "rate", "date"] )

# sg_items_set = set(sg_rating_df['itemId'].unique())

In [ ]:
!pip install torch dgl

In [ ]:
from collections import defaultdict
from collections import Counter
import dgl

# ca
# Constructing a mapping between userId and itemList
ca_item_mapping = {item: idx for idx, item in enumerate(ca_items_set)}
ca_user_mapping = {user: idx for idx, user in enumerate(ca_df['userId'].unique())}

# Create inverted index for each item
inverted_index = defaultdict(set)


# 使用一个Counter来记录每对用户之间的共同物品数量
pairwise_common_items_count = Counter()

for idx, row in ca_df.iterrows():
    user_idx = ca_user_mapping[row['userId']]
    items = eval(row['itemList'])
    for item in items:
        users_having_item = inverted_index[item]
        for other_user in users_having_item:
            if user_idx != other_user:
                # 为每一对用户的组合记录+1
                # 我们使用一个元组 (smaller_idx, larger_idx) 来确保唯一性
                pairwise_common_items_count[(min(user_idx, other_user), max(user_idx, other_user))] += 1
        inverted_index[item].add(user_idx)

# 通过检查计数器来收集边
src_nodes = []
dst_nodes = []

for (user1, user2), common_count in pairwise_common_items_count.items():
    if common_count >= 2:
        src_nodes.append(user1)
        dst_nodes.append(user2)

# 使用这些边来创建DGL图
g = dgl.graph((src_nodes, dst_nodes))


print("Number of nodes:", g.number_of_nodes())
print("Number of edges:", g.number_of_edges())


In [ ]:
import dgl
import dgl.nn as dglnn
import torch.nn as nn
import torch

# 定义GraphSAGE模型
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_layers):
        super(GraphSAGE, self).__init__()
        self.convs = nn.ModuleList()

        # 输入层
        self.convs.append(dglnn.SAGEConv(in_feats, h_feats, 'mean'))
        # 隐藏层
        for l in range(num_layers - 1):
            self.convs.append(dglnn.SAGEConv(h_feats, h_feats, 'mean'))

    def forward(self, g, in_feat):
        h = in_feat
        for conv in self.convs:
            h = conv(g, h)
        return h

# 根据您的图数据，您有397723个节点
node_count = 397723

# 为这些节点创建随机的5维输入特征
inputs = torch.randn(node_count, 5)

# 定义一个GraphSAGE模型实例
model = GraphSAGE(5, 16, 2)  # 5维输入特征，16维隐藏特征，2层

# 得到嵌入
g = dgl.graph((src_nodes, dst_nodes))

In [ ]:
!pip install scikit-learn

In [ ]:
# Cluster
import torch
import torch.nn.functional as F
from sklearn.cluster import KMeans

def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.t() / weight.sum(1)).t()

def Clustering(z, Uw):
    q = 1.0 / (1.0 + torch.sum(torch.pow(z.unsqueeze(1) - Uw, 2), 2))
    q = q.pow((1 + 1.0) / 2.0)
    q = (q.t() / torch.sum(q, 1)).t()
    p = target_distribution(q)
    kl_loss = F.kl_div(q.log(), p)
    return kl_loss, q


In [ ]:
# Initialize cluster centers Uw, here you can use KMeans or just randomly select some embeddings
# For simplicity, I am taking random embeddings. Let's assume we want 10 clusters
# Number of clusters
num_clusters = 10

# Number of clustering iterations
num_iterations = 500

embeddings = model(g, inputs)

# Apply K-Means clustering to initialize cluster centers
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(embeddings.detach().numpy())
Uw = torch.tensor(kmeans.cluster_centers_, requires_grad=True)

for iteration in range(num_iterations):
    embeddings = model(g, inputs)

    # Choose an optimizer
    optimizer = torch.optim.SGD([Uw], lr=0.01, momentum=0.9)

    # Zero gradients
    optimizer.zero_grad()

    # Forward pass: Compute q and KL loss
    kl_loss, ca_q = Clustering(embeddings, Uw)

    # Backward pass: Compute gradient of the loss with respect to model parameters
    kl_loss.backward(retain_graph=True)

    # Update Uw using gradient information
    optimizer.step()

    # Print the current KL loss value for monitoring
    if iteration % 100 == 0:
        print(f"Iteration {iteration}, KL Loss: {kl_loss.item()}")

In [ ]:
print(ca_q)

In [ ]:
ca_general_user = ca_q.mean(dim=0)
print(ca_general_user)

In [ ]:
with open("ca_general_user.txt", "w") as file:
    for value in ca_general_user:
        file.write(str(value.item()) + "\n")